In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.cluster import KMeans
import seaborn as sns
import matplotlib.pyplot as plt
import os
tf.config.list_physical_devices()
gpu = tf.config.list_physical_devices('GPU')
if gpu:
    # only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpu[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpu), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        print(e)
data = pd.read_csv("./dataset/acc_gyr.csv")
data

KeyboardInterrupt: 

In [6]:
def split_resampling(data, seed=1218):
    new_data = data.drop("label",axis=1).to_numpy().reshape(-1, 400, 6)
    labels = np.array(data.label.iloc[np.arange(0,data.shape[0], 400)])

    sm = SMOTE(random_state=seed)
    y = labels
    X, y = sm.fit_resample(new_data.reshape(-1, 400*6), y)

    # split
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    # scale
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train.reshape(-1, 400*6))
    X_test = scaler.fit_transform(X_test.reshape(-1, 400*6))

    # reshape
    X_train = X_train.reshape(-1, 400, 6)
    X_test = X_test.reshape(-1, 400, 6)
    input_shape = X_train.shape[1:]
    seq_len, n_features = input_shape

    le = LabelEncoder()
    le.fit(np.unique(y_train))
    y_train = le.transform(y_train)
    y_test = le.transform(y_test)
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=7)
    y_test = tf.keras.utils.to_categorical(y_test, num_classes=7)

    X_train = X_train.reshape(-1, seq_len, n_features)
    X_test = X_test.reshape(-1, seq_len, n_features)

    return X_train, X_test, y_train, y_test, input_shape
X_train, X_test, y_train, y_test, input_shape = split_resampling(data)
X_train.shape

(330, 400, 6)

In [ ]:
model = models.Sequential([
    layers.Conv1D(16, kernel_size=3, activation='relu', input_shape=input_shape),  # 16 filter nhỏ hơn 32
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(16, kernel_size=3, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.GlobalAveragePooling1D(),  # thay Flatten bằng GlobalAveragePooling1D
    layers.Dense(16, activation='relu'),  # Dense nhỏ hơn
    layers.Dense(7, activation='softmax')  # softmax cho phân lớp đa lớp
    ])

model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(X_train, y_train, epochs=50, batch_size=8, validation_data=(X_test, y_test))


Model: "sequential_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_43 (Conv1D)              │ (None, 398, 32)        │           608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_41 (MaxPooling1D) │ (None, 199, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_19 (Flatten)            │ (None, 6368)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_52 (Dense)                │ (None, 64)             │       407,616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_53 (Dense)                │ (None, 7)              │           455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 408,679 (1.56 MB)

 Trainable params: 408,679 (1.56 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.5739 - loss: 1.3677 - val_accuracy: 0.8378 - val_loss: 0.6431
Epoch 2/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9917 - loss: 0.0882 - val_accuracy: 0.9640 - val_loss: 0.1891
Epoch 3/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.0086 - val_accuracy: 0.9459 - val_loss: 0.1763
Epoch 4/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 0.0052 - val_accuracy: 0.9640 - val_loss: 0.1568
Epoch 5/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.0024 - val_accuracy: 0.9640 - val_loss: 0.1562
Epoch 6/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 1.0000 - loss: 0.0020 - val_accuracy: 0.9640 - val_loss: 0.1543
Epoch 7/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.0016 - val_accuracy: 0.9640 - val_loss: 0.1598
Epoch 8/50
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.0011 - val_accuracy: 0.9640 - val_loss

In [ ]:
# Convert Keras model to a tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
tflite_model_name='model'
open(tflite_model_name + '.tflite', 'wb').write(tflite_model)

In [ ]:
# Function: Convert some hex value into an array for C programming
def hex_to_c_array(hex_data, var_name):

  c_str = ''

  # Create header guard
  c_str += '#ifndef ' + var_name.upper() + '_H\n'
  c_str += '#define ' + var_name.upper() + '_H\n\n'

  # Add array length at top of file
  c_str += '\nunsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

  # Declare C variable
  c_str += 'unsigned char ' + var_name + '[] = {'
  hex_array = []
  for i, val in enumerate(hex_data) :

    # Construct string from hex
    hex_str = format(val, '#04x')

    # Add formatting so each line stays within 80 characters
    if (i + 1) < len(hex_data):
      hex_str += ','
    if (i + 1) % 12 == 0:
      hex_str += '\n '
    hex_array.append(hex_str)

  # Add closing brace
  c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

  # Close out header guard
  c_str += '#endif //' + var_name.upper() + '_H'

  return c_str